In [11]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
import os
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
from amftrack.pipeline.launching.run_super import run_parallel



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230123_Plate530/"
upload_targ = "/DATA/FLUORESCENCE/DATA_NileRed/20230123_Plate530/Analysis/"

In [59]:
test_name = glob(directory_targ + '*')[0]
imgs_address = test_name
print(test_name)

/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230123_Plate530/20230123_Plate530_002


In [60]:

plate_list = glob(directory_targ + "*_Plate*_*")
datadict = {'address_total' : plate_list}
dataframe = pd.DataFrame(data=datadict)
dataframe['video_name'] = [address.split('/')[-1] for address in dataframe['address_total']]
dataframe['plate_nr'] = [address.split('/')[-1].split('_')[-2][5:] for address in dataframe['address_total']]
dataframe['video_nr'] = [int(address.split('/')[-1].split('_')[-1]) for address in dataframe['address_total']]
dataframe['date_imaged'] = [address.split('/')[-1][:8] for address in dataframe['address_total']]

parent_folder = []
data_table = []
magnification = []
fps = []
mode = []

for row in dataframe.iloc:
#     print(row)
    parent_folder.append(str(Path(row['address_total']).parent))
    excel_file = glob(str(Path(row['address_total']).parent) + f'/*{row["date_imaged"]}*{row["plate_nr"]}.xl*')
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
        excel_table = pd.read_excel(excel_file[0])
#         print(row['video_name'])
#         print(excel_table['Unnamed: 0'][21])
#         print(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'])
        magnification.append(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'].iloc[0])
        fps.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['FPS'].iloc[0])
        mode.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Bright-field (BF)\nor\nFluorescence (F)'].iloc[0])
        
    else:
        csv_file = glob(str(Path(row['address_total']).parent) + f'/{row["date_imaged"]}*{row["plate_nr"]}.csv')
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
            df_comma = pd.read_csv(csv_file[0], nrows=1,sep=",")
            df_semi = pd.read_csv(csv_file[0], nrows=1, sep=";")
            if df_comma.shape[1]>df_semi.shape[1]:
                csv_table = pd.read_csv(csv_file[0], sep=",")
            else:
                csv_table = pd.read_csv(csv_file[0], sep=";")
            magnification.append(csv_table[csv_table['video']==int(row['video_name'].split("_")[-1])]['Lens'].iloc[0])
            fps.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['fps'].iloc[0])
            mode.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['Illumination'].iloc[0])

        else:
            print("Halp! No datatable found! Pls add the excel file to the folders.")

dataframe['fps'] = fps
dataframe['magnification'] = magnification
dataframe['mode'] = mode
dataframe['parent_folder'] = parent_folder
dataframe['data_table'] = data_table

dataframe = dataframe.sort_values(by='video_name')
# dataframe = dataframe[dataframe['video_nr']== 5]
# dataframe = dataframe[dataframe['date_imaged'] == '20221026']
# print(len(dataframe))


if dataframe.isnull().values.any():
    print("Found NaNs in the excel files! Blame the experimentalists.")
    dataframe = dataframe.interpolate(method='pad', limit_direction='forward')
if dataframe.isnull().values.any():
    raise("This excel sheet is unworkable, please ask the responisble person")
dataframe

,address_total,video_name,plate_nr,video_nr,date_imaged,fps,magnification,mode,parent_folder,data_table
2,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_001,530,1,20230123,10.0,4.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
0,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_002,530,2,20230123,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
95,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_003,530,3,20230123,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
70,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_004,530,4,20230123,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
3,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_005,530,5,20230123,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
...,...,...,...,...,...,...,...,...,...,...
12,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_097,530,97,20230123,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
21,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_098,530,98,20230123,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
72,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_099,530,99,20230123,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
23,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20230123_Plate530_100,530,100,20230123,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...


In [61]:
# FINAL CHECK FOR SOURCE FOLDER AND UPLOAD FOLDER

# Please make sure that the upload folder is correct, 
# as the program WILL overwrite that which is already there.
print(directory_targ)
print(upload_targ)

/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230123_Plate530/
/DATA/FLUORESCENCE/DATA_NileRed/20230123_Plate530/Analysis/


In [62]:
nr_parallel = np.min([len(dataframe.index), 16])

run_parallel(
    "flux_extract.py",
    [directory_targ, 15, 0.95, 0.001, 50, upload_targ],
    dataframe,
    nr_parallel,
    "1:00:00",
    "flux_extract"
)

/home/svstaalduine/bash/job.sh
Sending jobs with id 1685023637507765840
101
Submitted batch job 2799546


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2799547


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2799548


...

Submitted batch job 2799549


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2799550


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2799551
